In [415]:
# requirements
import os
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk

In [416]:
'''
This section is designed to optimise image pre-processing 
Aims:
- manage sequential images and folder structure such that sequence is easily retained
- perform visual/mathematical filtering to refine image contrast and enable the subsequent algorithm
- denoising the image (speckle decorrelation?)
- contrast enhancement
- motion artifact correction?
'''
# input variable is a folder containing png images in chronological order for now.
def preprocess_image_sequence(image_sequence):

    # create a dir for sequential preprocessed images
    if not os.path.exists("denoised_sequence"):
        os.makedirs("denoised_sequence")
    if not os.path.exists("contrasted_sequence"):
        os.makedirs("contrasted_sequence")
    if not os.path.exists("preprocessed_sequence"):
        os.makedirs("preprocessed_sequence")

    # sort image paths from input folder
    image_paths = sorted(glob(os.path.join(image_sequence, "*.png")))

    # read first image as the reference image
    reference_image = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)

    for idx, image_path in enumerate(image_paths):
        # read in current image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # denoise image via median filtering 
        denoised_image = cv2.medianBlur(image, 3)
        output_path = os.path.join("denoised_sequence", f"{idx:03d}.png")
        cv2.imwrite(output_path, denoised_image)

        # contrast enhancement via adaptive histogram equalisation
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced_image = clahe.apply(denoised_image)
        output_path = os.path.join("contrasted_sequence", f"{idx:03d}.png")
        cv2.imwrite(output_path, enhanced_image)

        # motion artifact correction or rigid alignment of current image to the reference image
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        _, warp_matrix = cv2.findTransformECC(reference_image, enhanced_image, warp_matrix, cv2.MOTION_EUCLIDEAN)

        corrected_image = cv2.warpAffine(enhanced_image, warp_matrix, (enhanced_image.shape[1], enhanced_image.shape[0]),
                                         flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        output_path = os.path.join("preprocessed_sequence", f"{idx:03d}.png")
        cv2.imwrite(output_path, corrected_image)

    # Visualize the steps
    fig, axes = plt.subplots(1, 4, figsize=(16, 5))
    images = [image, denoised_image, enhanced_image, corrected_image]
    titles = ['Original', 'Denoised', 'Contrast Enhanced', 'Motion Corrected']

    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap='gray')
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.savefig("preprocessing_pipeline")

# a simplified single image process for utilisation by the tkinter 
def preprocess_single_image(img, reference_img=None):
    steps = {}

    # Step 1: Original
    steps['Original'] = img.copy()

    # Step 2: Denoising
    denoised = cv2.medianBlur(img, 3)
    steps['Denoised'] = denoised

    # Step 3: Contrast Enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)
    steps['Contrast Enhanced'] = enhanced

    # Step 4: Motion Artifact Correction
    if reference_img is not None:
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        try:
            _, warp_matrix = cv2.findTransformECC(reference_img, enhanced, warp_matrix, cv2.MOTION_EUCLIDEAN)
            corrected = cv2.warpAffine(enhanced, warp_matrix, (enhanced.shape[1], enhanced.shape[0]),
                                       flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        except cv2.error:
            corrected = enhanced.copy()  # fallback if ECC fails
    else:
        corrected = enhanced.copy()

    steps['Motion Corrected'] = corrected

    return steps



In [417]:
'''
The purpose of this section is to test combinations of several tracking algorithms into a single, hybrid methodology
Aims:
- building a combined hybrid methodology which factors the strengths of each algorithm
- tweaking the influence or 'strength' of each algorithm based on overall model performance
- perhaps model efficiency optimisation
'''

# optical tracking implementation
'''
Meant to target larger homogeneous regions of the tissue (areas of similar contrast)
'''
def optical_tracking():
    return 0

# feature based alignment implementation
'''
Meant to target tissue landmarks and structures
need to test some segmentation algorithms and maybe combine to form an OCT optimal strategy
'''
def feature_alignment():
    return 0

# deep learning fine-tuning tracking implementation
'''
Uses a library of labelled imagery to make tracking adjustments
not sure yet on method of implementation, probably looking at CNN
'''
def CNN():
    return 0



In [418]:
# === Base Page Class ===
class Page(tk.Frame):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)

# === Individual Pages ===
class HomePage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        label = ttk.Label(self, text="Welcome to the OCT Deformation Tracking Suite", font=("Helvetica", 18))
        label.pack(pady=20)


In [419]:
class PreprocessingPage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        # Controls
        controls = ttk.Frame(self)
        controls.pack(fill='x', pady=10)
        ttk.Button(controls, text="Select Folder", command=self.load_images).pack(side='left', padx=5)
        ttk.Button(controls, text="Previous", command=self.prev_image).pack(side='left', padx=5)
        ttk.Button(controls, text="Next", command=self.next_image).pack(side='left', padx=5)
        # Canvas
        self.canvas = tk.Canvas(self, bg='#f0f0f0')
        self.canvas.pack(fill='both', expand=True, padx=10, pady=10)
        # State
        self.images = []
        self.reference_img = None
        self.index = 0
        self.preprocessed_steps = []
        self.tk_imgs = {}

    def preprocess_single_image(self, img, reference_img=None):
        steps = {'Original': img.copy()}
        denoised = cv2.medianBlur(img, 3); steps['Denoised'] = denoised
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(denoised); steps['Contrast Enhanced'] = enhanced
        if reference_img is not None:
            warp_matrix = np.eye(2,3, dtype=np.float32)
            try:
                _, warp_matrix = cv2.findTransformECC(reference_img, enhanced, warp_matrix, cv2.MOTION_EUCLIDEAN)
                corrected = cv2.warpAffine(enhanced, warp_matrix, (enhanced.shape[1], enhanced.shape[0]),
                                           flags=cv2.INTER_LINEAR+cv2.WARP_INVERSE_MAP)
            except cv2.error:
                corrected = enhanced.copy()
        else:
            corrected = enhanced.copy()
        steps['Motion Corrected'] = corrected
        return steps

    def load_images(self):
        folder = filedialog.askdirectory()
        if not folder:
            return
        paths = sorted(glob(folder + "/*.png"))
        self.images = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in paths]
        self.reference_img = self.images[0] if self.images else None
        self.index = 0
        self.preprocessed_steps = [self.preprocess_single_image(img, self.reference_img) for img in self.images]
        self.show_current_image()

    def show_current_image(self):
        self.canvas.delete("all")
        if not self.preprocessed_steps:
            return
        steps = self.preprocessed_steps[self.index]
        titles = list(steps.keys())
        for i, title in enumerate(titles):
            img = steps[title]
            img = Image.fromarray(img).resize((220, 220))
            tk_img = ImageTk.PhotoImage(img)
            self.tk_imgs[i] = tk_img
            x = 10 + i * 240
            self.canvas.create_image(x, 10, anchor='nw', image=tk_img)
            self.canvas.create_text(x + 110, 240, text=title, font=("Helvetica", 12))
        self.canvas.update()

    def next_image(self):
        if self.index < len(self.images) - 1:
            self.index += 1
            self.show_current_image()
    def prev_image(self):
        if self.index > 0:
            self.index -= 1
            self.show_current_image()

In [420]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk

class OpticalFlowPage(ttk.Frame):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)

        # Available algorithms
        self.algorithms = ['Farneback', 'Lucas-Kanade', 'Speckle Tracking', 'TVL1']
        try:
            self.tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
        except Exception:
            self.algorithms.remove('TVL1')
            self.tvl1 = None

        # Controls
        ctrl = ttk.Frame(self)
        ctrl.pack(fill='x', pady=10)

        # State
        self.images = []
        self.index  = 0
        self.tk_imgs = []

        ttk.Button(ctrl, text="Select Folder", command=self.load_images).pack(side='left', padx=5)
        ttk.Button(ctrl, text="Previous Image", command=self.prev_image).pack(side='left', padx=5)
        ttk.Button(ctrl, text="Next Image", command=self.next_image).pack(side='left', padx=5)

        ttk.Label(ctrl, text="Algorithm:").pack(side='left', padx=(20,5))
        self.selected_alg = tk.StringVar(value=self.algorithms[0])
        alg_combo = ttk.Combobox(ctrl, textvariable=self.selected_alg,
                                 values=self.algorithms, state='readonly', width=15)
        alg_combo.pack(side='left', padx=5)
        alg_combo.bind('<<ComboboxSelected>>', lambda e: self.show_flow())

        ttk.Label(ctrl, text="Grid Step:").pack(side='left', padx=(20,5))
        self.grid_step = tk.IntVar(value=20)
        step_slider = ttk.Scale(ctrl, from_=5, to=50, variable=self.grid_step,
                                command=lambda e: self.show_flow())
        step_slider.pack(side='left', padx=5)
        self.step_label = ttk.Label(ctrl, text=str(self.grid_step.get()))
        self.step_label.pack(side='left')
        self.grid_step.trace_add("write", lambda *args: self.step_label.config(text=str(self.grid_step.get())))

        self.show_net_var  = tk.BooleanVar(value=True)
        self.show_inc_var  = tk.BooleanVar(value=True)
        self.color_net_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(ctrl, text="Show Net Vector",         variable=self.show_net_var,  command=self.show_flow).pack(side='left', padx=(20,5))
        ttk.Checkbutton(ctrl, text="Show Incremental Vector", variable=self.show_inc_var,  command=self.show_flow).pack(side='left', padx=5)
        ttk.Checkbutton(ctrl, text="Color Net by Mag",        variable=self.color_net_var, command=self.show_flow).pack(side='left', padx=5)

        # Canvas for visualization
        self.canvas = tk.Canvas(self, bg='#f0f0f0')
        self.canvas.pack(fill='both', expand=True, padx=10, pady=10)

        

    def load_images(self):
        folder = filedialog.askdirectory()
        if not folder:
            return
        import os, glob
        self.image_paths = sorted(glob.glob(os.path.join(folder, "*.png")))
        self.images      = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in self.image_paths]
        self.index       = 0
        self.show_flow()

    def preprocess_variants(self, img):
        variants = {
            "Raw":      img.copy(),
            "Denoised": cv2.medianBlur(img, 3)
        }
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        variants["Enhanced"] = clahe.apply(variants["Denoised"])
        return variants

    def compute_flow(self, img1, img2, method):
        if method == 'Farneback':
            return cv2.calcOpticalFlowFarneback(
                img1, img2, None,
                pyr_scale=0.5, levels=3,
                winsize=15, iterations=3,
                poly_n=5, poly_sigma=1.2,
                flags=0
            )
        if method == 'TVL1' and self.tvl1:
            return self.tvl1.calc(img1, img2, None)
        if method == 'Lucas-Kanade':
            p0 = cv2.goodFeaturesToTrack(
                img1, maxCorners=200,
                qualityLevel=0.01, minDistance=7, blockSize=7
            )
            if p0 is None:
                return None
            p1, st, _ = cv2.calcOpticalFlowPyrLK(
                img1, img2, p0, None,
                winSize=(15,15), maxLevel=2
            )
            pts0 = p0[st.flatten()==1].reshape(-1,2)
            pts1 = p1[st.flatten()==1].reshape(-1,2)
            return (pts0, pts1)
        if method == 'Speckle Tracking':
            h, w = img1.shape
            pts = np.array([
                [x, y]
                for y in range(0, h, self.grid_step.get())
                for x in range(0, w, self.grid_step.get())
            ])
            pts1 = []
            half_t = 5
            tpl_h, tpl_w = 2*half_t+1, 2*half_t+1
            s = self.grid_step.get()
            for x0,y0 in pts:
                tpl = img1[
                    max(y0-half_t,0):y0+half_t+1,
                    max(x0-half_t,0):x0+half_t+1
                ]
                win = img2[
                    max(y0-s,0):y0+s+1,
                    max(x0-s,0):x0+s+1
                ]
                if win.shape[0]<tpl_h or win.shape[1]<tpl_w:
                    pts1.append((x0,y0))
                    continue
                res = cv2.matchTemplate(win, tpl, cv2.TM_CCOEFF_NORMED)
                _,_,_,max_loc = cv2.minMaxLoc(res)
                dx = max_loc[0] - (win.shape[1]//2 - half_t)
                dy = max_loc[1] - (win.shape[0]//2 - half_t)
                pts1.append((x0+dx, y0+dy))
            return (pts, np.array(pts1))
        return None

    def draw_arrows(self, vis, flow, method, color):
        step = self.grid_step.get()
        if method in ['Farneback','TVL1']:
            h,w = vis.shape[:2]
            y,x = np.mgrid[step//2:h:step, step//2:w:step].astype(int)
            fx,fy = flow[y,x].T
            for x0,y0,dx,dy in zip(x.flatten(),y.flatten(),fx.flatten(),fy.flatten()):
                cv2.arrowedLine(
                    vis, (x0,y0), (int(x0+dx),int(y0+dy)),
                    color=color, thickness=1
                )
        else:
            pts0,pts1 = flow
            for (x0,y0),(x1,y1) in zip(pts0,pts1):
                cv2.arrowedLine(
                    vis, (int(x0),int(y0)), (int(x1),int(y1)),
                    color=color, thickness=1
                )

    def _draw_arrows_from_pts(self, vis, pts0, disp, color_or_lut):
        """
        Draw arrows from pts0 with displacement disp.
        color_or_lut: BGR tuple for uniform color or LUT (256×1×3) for JET mapping.
        """
        if isinstance(color_or_lut, tuple):
            for (x0,y0),(dx,dy) in zip(pts0, disp):
                cv2.arrowedLine(vis,
                                (int(x0),int(y0)),
                                (int(x0+dx),int(y0+dy)),
                                color_or_lut, thickness=1)
        else:
            lut = color_or_lut[:,0]  # shape (256,3)
            mags = np.hypot(disp[:,0], disp[:,1])
            maxm = mags.max() or 1.0
            for (x0,y0),(dx,dy),m in zip(pts0, disp, mags):
                idx = int(255 * min(m/maxm, 1.0))
                b,g,r = lut[idx]
                cv2.arrowedLine(vis,
                                (int(x0),int(y0)),
                                (int(x0+dx),int(y0+dy)),
                                (int(b),int(g),int(r)), thickness=1)

    def show_flow(self):
        self.canvas.delete("all")
        self.tk_imgs.clear()
        if not self.images:
            return

        alg       = self.selected_alg.get()
        curr      = self.images[self.index]
        prev      = self.images[self.index-1] if self.index>0 else curr
        ref       = self.images[0]
        show_net  = self.show_net_var.get()
        show_inc  = self.show_inc_var.get()
        color_net = self.color_net_var.get()

        curr_vars = self.preprocess_variants(curr)
        prev_vars = self.preprocess_variants(prev)
        ref_vars  = self.preprocess_variants(ref)

        # Precompute net flows & displacements
        net_disps = {}
        maxima    = []
        for var in ["Raw","Denoised","Enhanced"]:
            f = self.compute_flow(ref_vars[var], curr_vars[var], alg)
            if f is None:
                net_disps[var] = (np.zeros((0,2),int), np.zeros((0,2),float))
                maxima.append(0.0)
            elif isinstance(f, tuple):
                p0,p1 = f
                disp = p1 - p0
                net_disps[var] = (p0.astype(int), disp)
                maxima.append(np.linalg.norm(disp,axis=1).max() if len(disp)>0 else 0.0)
            else:  # dense
                step = self.grid_step.get()
                h,w = f.shape[:2]
                y,x = np.mgrid[step//2:h:step, step//2:w:step].astype(int)
                pts0 = np.stack([x.flatten(), y.flatten()],axis=-1)
                fx,fy = f[...,0], f[...,1]
                disp = np.stack([fx[y,x], fy[y,x]],axis=-1).reshape(-1,2)
                net_disps[var] = (pts0, disp)
                maxima.append(np.hypot(disp[:,0],disp[:,1]).max() if disp.size else 0.0)

        global_max = max(maxima) or 1.0
        jet_lut    = cv2.applyColorMap(np.arange(256,dtype=np.uint8), cv2.COLORMAP_JET)

        # Canvas sizing
        self.canvas.update_idletasks()
        W   = self.canvas.winfo_width()
        pad = 20
        tw  = (W - pad*4)//3

        for idx,var in enumerate(["Raw","Denoised","Enhanced"]):
            vis = cv2.cvtColor(curr_vars[var], cv2.COLOR_GRAY2BGR)

            # Incremental (green)
            if show_inc:
                f_inc = self.compute_flow(prev_vars[var], curr_vars[var], alg)
                if f_inc is not None:
                    self.draw_arrows(vis, f_inc, alg, (0,255,0))

            # Net
            if show_net:
                p0, disp = net_disps[var]
                if p0.size:
                    if color_net:
                        self._draw_arrows_from_pts(vis, p0, disp, jet_lut)
                    else:
                        self._draw_arrows_from_pts(vis, p0, disp, (0,0,255))

            # Thumbnail & display
            h2,w2 = vis.shape[:2]
            th    = int(tw * h2 / w2)
            thumb = cv2.resize(vis, (tw,th), interpolation=cv2.INTER_AREA)
            img_tk = ImageTk.PhotoImage(
                        Image.fromarray(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB)))
            self.tk_imgs.append(img_tk)

            x = pad + idx*(tw+pad)
            y = pad
            self.canvas.create_image(x, y, anchor='nw', image=img_tk)
            self.canvas.create_text(x+tw//2, y+th+10,
                                    text=var, font=("Helvetica",8,"bold"))

        self.master.master.title(
            f"{alg} Vectors — Img0→Img{self.index}/{len(self.images)-1}"
        )

    def next_image(self):
        if self.index < len(self.images)-1:
            self.index += 1
            self.show_flow()

    def prev_image(self):
        if self.index > 0:
            self.index -= 1
            self.show_flow()


In [421]:
class FeatureRegistrationPage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        ttk.Label(self, text="Feature-Based Registration", font=("Helvetica", 18)).pack(pady=20)



In [422]:
class DeepLearningRegistrationPage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        ttk.Label(self, text="Deep Learning Registration", font=("Helvetica", 18)).pack(pady=20)



In [423]:
class HybridModelPage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        ttk.Label(self, text="Hybrid Model (Feature + Flow + Deep Learning)", font=("Helvetica", 18)).pack(pady=20)



In [424]:
class BenchmarkingPage(Page):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        ttk.Label(self, text="Benchmarking Framework", font=("Helvetica", 18)).pack(pady=20)



In [ ]:
# === Main Application ===
class MainApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("OCT Deformation Tracking Suite")
        self.geometry("1200x800")
        style = ttk.Style(self)
        style.theme_use('clam')
        style.configure('TNotebook.Tab', padding=(10, 10))

        notebook = ttk.Notebook(self)
        notebook.pack(fill='both', expand=True)

        # Add pages to notebook
        for PageClass, title in [
            (HomePage, "Home"),
            (PreprocessingPage, "Preprocessing"),
            (OpticalFlowPage, "Optical Flow"),
            (FeatureRegistrationPage, "Feature Reg."),
            (DeepLearningRegistrationPage, "Deep Learning"),
            (HybridModelPage, "Hybrid Model"),
            (BenchmarkingPage, "Benchmarking"),
        ]:
            page = PageClass(notebook)
            notebook.add(page, text=title)

if __name__ == "__main__":
    MainApp().mainloop()